In [ ]:
# Assignment 13: Load pre-trained VGG16 for Feature Extraction

In [ ]:
# Imports
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model

# Load pre-trained VGG16 for Feature Extraction
base_model = VGG16(weights='imagenet', include_top=True, input_tensor=Input(shape=(224, 224, 3)))

# Create a new model for feature extraction (excluding top layers)
feature_extractor = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)

# Set feature extractor layers as not trainable
for layer in feature_extractor.layers:
    layer.trainable = False

# Create a new model for classification on top of the feature extractor
x = feature_extractor.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x) # 10 classes for MNIST

model = Model(inputs=feature_extractor.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,140,042 (80.64 MB)

 Trainable params: 6,425,354 (24.51 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the images
def preprocess_image(image):
    # Add a channel dimension (from 28, 28 to 28, 28, 1)
    image = tf.expand_dims(image, axis=-1)
    # Resize to 224x224
    image = tf.image.resize(image, (224, 224))
    # Convert to 3 channels
    image = tf.image.grayscale_to_rgb(image)
    # Expand dimensions to add batch size
    image = tf.expand_dims(image, axis=0)
    # Preprocess for VGG16
    image = tf.keras.applications.vgg16.preprocess_input(image)
    return image

# Apply preprocessing to a subset of the data for demonstration
# Note: Processing the entire MNIST dataset this way might take a long time
# Consider using a smaller subset or a data generator for larger datasets
num_samples = 1000 # Process a smaller subset
x_train_processed = tf.concat([preprocess_image(x_train[i]) for i in range(num_samples)], axis=0)
x_test_processed = tf.concat([preprocess_image(x_test[i]) for i in range(num_samples)], axis=0)

print("Shape of processed training data:", x_train_processed.shape)
print("Shape of processed testing data:", x_test_processed.shape)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Shape of processed training data: (1000, 224, 224, 3)
Shape of processed testing data: (1000, 224, 224, 3)


In [ ]:
# Train the model
# Using a smaller subset of data for demonstration purposes as processing the full dataset is time consuming
history = model.fit(x_train_processed, y_train[:num_samples], epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 193ms/step - accuracy: 1.0000 - loss: 1.4084e-04 - val_accuracy: 0.9600 - val_loss: 0.5485
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 187ms/step - accuracy: 1.0000 - loss: 1.0450e-05 - val_accuracy: 0.9550 - val_loss: 0.5731
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 189ms/step - accuracy: 1.0000 - loss: 6.9665e-06 - val_accuracy: 0.9550 - val_loss: 0.5748
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 189ms/step - accuracy: 1.0000 - loss: 4.9266e-06 - val_accuracy: 0.9550 - val_loss: 0.5747
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 206ms/step - accuracy: 1.0000 - loss: 2.8822e-06 - val_accuracy: 0.9550 - val_loss: 0.5745


In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test_processed, y_test[:num_samples])

print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 146ms/step - accuracy: 0.9516 - loss: 0.3608
Test Loss: 0.5063265562057495
Test Accuracy: 0.9419999718666077
